<a href="https://colab.research.google.com/github/saptarshi7karmakar/LLMExplore/blob/main/collab_notes/crew_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
!pip install --q crewai
!pip install --q -U duckduckgo-search
!pip install --q langchain_google_genai
!pip install --q langchain-community
!pip install --q crewai-tools

In [36]:
# !pip install --q crewai-tools

In [186]:
from crewai import Agent, LLM
from google.colab import userdata
from litellm import completion

# Switched to LiteLM Standards to ensure Tools Compatibility
# gemini-pro
# Gemini 2.0 Flash-Lite

llm_chat = LLM(
    api_key=userdata.get('GOOGLE_API_KEY'),
    model="gemini/gemini-pro",
)
llm_chat

In [189]:
llm_chat.get_context_window_size()

6963

In [177]:
# from langchain_google_genai import ChatGoogleGenerativeAI
# from google.colab import userdata
# from litellm import completion

# # os.environ['GEMINI_API_KEY'] = userdata.get('GOOGLE_API_KEY')

# mod = "gemini/gemini-pro"
# # mod = "google/gemma-3-1b-it"
# # completion(model="gemini/gemini-pro")
# llm_chat = ChatGoogleGenerativeAI(model=my_llm,
#                                   verbose = True,
#                                   temperature = 0.6,
#                                   google_api_key = userdata.get('GOOGLE_API_KEY'))

In [190]:
llm_chat

In [179]:
import os
import uuid
from crewai import Agent, Task, Crew, Process

### Simple Search and Analyze Use Cases

In [191]:
# Custom Search Tool from Base Tools to workaround the dependency issue
from langchain.tools import DuckDuckGoSearchRun
from crewai.tools import BaseTool

class MyCustomDuckDuckGoTool(BaseTool):
    name: str = "DuckDuckGo Search Tool"
    description: str = "Search the web for a given query."

    def _run(self, query: str) -> str:
        # Ensure the DuckDuckGoSearchRun is invoked properly.
        duckduckgo_tool = DuckDuckGoSearchRun()
        response = duckduckgo_tool.invoke(query)
        return response

    def _get_tool(self):
        # Create an instance of the tool when needed
        return MyCustomDuckDuckGoTool()

search_tool = MyCustomDuckDuckGoTool()
search_tool_for_evaluator = MyCustomDuckDuckGoTool()

In [192]:
research_agent = Agent(
                       name = "Research Agent",
                       role = "Senior Researcher",
                       goal = "Uncover most useful parameters in Open Table Formats",
                       backstory = """You work at a leading tech company.
                       Your expertise is in identifying all influencial parameters from the industruy for technologies in Data Intensive Engineering Space.
                       You have a knack for dissecting complex data and presenting""",
                       verbose=True,
                       allow_delegation=False,
                       tools = [search_tool],
                       llm = llm_chat)


In [193]:
decision_agent = Agent(
                       name = "Analysis & Decision Agent",
                       role = "Senior Analyst & Decision Maker",
                       goal = "Analyze and Craft Recommendation with compelling reasoning",
                       backstory = """"You are a Industry trend analyst, known for depth of analysis and data driven recommendations.
                       you analyze variety of aspects of the product to come up with compelling recommendations""",
                       verbose=True,
                       allow_delegation=False,
                       tools = [],
                       llm = llm_chat
)

evaluator_agent = Agent(
                       name = "Recommender Agent",
                       role = "Leader",
                       goal = "Understand, Review,  Cleanse the Information to craft final recommendation",
                       backstory = """"You are the senior technology leader, who is expert in converging to final decision based on options.
                       You review different proposals to curve out most impactful decision based on current org and industry understanding""",
                       verbose=True,
                       allow_delegation=False,
                       tools = [search_tool_for_evaluator],
                       llm = llm_chat
)

In [194]:
# Create tasks for the agents

task1 = Task(
  description="""Conduct a comprehensive analysis of the latest advancements in Opentable Formats in the industry such as Delta Lake, Iceberg and others.
  Identify key parameters, applications on on premises and cloud set up, and potential industry future directions.
  Your final answer MUST be a table with all collected parameters and possible data points""",
  expected_output="A detailed table summarizing key parameters, applications, and future directions of Open Table Formats.",
  agent=research_agent
)

task2 = Task(
    description="""Using the information provided in the table format, run deep research and analysis to come up with recommendation with suitable ranking.
    the analysis should suggest ranking for the use cases where all the systems for the target use cases are on premises infrastructure.
    the final report should contain top 5 picks with deep down data points and decisioning parameters and visualisations on the considered data.
    """,
    expected_output="A report providing ranked recommendations for Open Table Formats in on-premises environments, including data points and decisioning parameters.",
    agent=decision_agent
)

task3 = Task(
    description="""
    Review the final information provided with ranks, recommendations.
    Deep dive into the options to tally against the industry trends and current organisational need to build a robust on premises solution.
    Cleanse all the information to depict inly the relevant recommendation.""",
    expected_output="A crisp well structured report with top relevant options with reasoning.",
    agent=evaluator_agent
)

In [195]:
# Crew definition and in action.
opentable_crew = Crew(
    name="OpenTable Crew",
    agents=[research_agent, decision_agent, evaluator_agent],
    tasks=[task1, task2, task3],
    verbose=True,
    process=Process.sequential,
    max_rpm=15 # Introduced to not rate limit for the entire crew
)
opentable_crew

Crew(id=08ac2cd3-1169-4aa1-b344-8e7db530bdcb, process=Process.sequential, number_of_agents=3, number_of_tasks=3)

In [197]:
opentable_crew.check_config()

Crew(id=08ac2cd3-1169-4aa1-b344-8e7db530bdcb, process=Process.sequential, number_of_agents=3, number_of_tasks=3)

In [198]:
opentable_crew.kickoff()

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: OpenTable Crew                                                                                           │
│  ID: 08ac2cd3-1169-4aa1-b344-8e7db530bdcb                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Researcher                                                                                       │
│                                                                                                                 │
│  Task: Conduct a comprehensive analysis of the latest advancements in Opentable Formats in the industry such    │
│  as Delta Lake, Iceberg and others.                                                                             │
│    Identify key parameters, applications on on premises and cloud set up, and potential industry future         │
│  directions.                                                                                                    │
│    Your final answer MUST be a table with all collected parameters and possible data points                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new

LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.

...

ERROR:root:LiteLLM call failed: litellm.NotFoundError: VertexAIException - {
  "error": {
    "code": 404,
    "message": "models/gemini-pro is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.",
    "status": "NOT_FOUND"
  }
}




 An unknown error occurred. Please check the details below.



╭───────────────────────────────────────────────── Task Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Failed                                                                                                    │
│  Name: c868819f-8a42-48c7-9d3f-e08c6e124161                                                                     │
│  Agent: Senior Researcher                                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Crew Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Failed                                                                                          │
│  Name: OpenTable Crew                                                                                           │
│  ID: 08ac2cd3-1169-4aa1-b344-8e7db530bdcb                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output:                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

NotFoundError: litellm.NotFoundError: VertexAIException - {
  "error": {
    "code": 404,
    "message": "models/gemini-pro is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.",
    "status": "NOT_FOUND"
  }
}
